# 8.2 Deep Dream

- try to maximize the activation of entire layers (mixing together visualizations of large numbers of features at onec)
- start not from blank, slightly noisy input, but rather from an existing imgae - thus the resulting effects latch on to preexisting visual patterns, distorting elements of the image in a somewhat artistic fashion.
- The input images are processed at different scales (called octaves), which improves the quality of the visualizations.

In [ ]:
# ted talk : https://www.ted.com/talks/blaise_aguera_y_arcas_how_computers_are_learning_to_be_creative?language=en#t-898371


In [3]:
# 8.8 Loding the pretrained Inception V3 model

from keras.applications import inception_v3
from keras import backend as K
K.set_learning_phase(0)

model = inception_v3.InceptionV3(weights = 'imagenet',include_top=False)

87916544/87910968 [==============================] - 56s 1us/step


In [12]:
from keras.utils import plot_model
plot_model(model,show_shapes=True,to_file="inceptionv3.png")

In [5]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 3 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [61]:
# 8.9 Setting up the DeepDream configuration

layer_contributions = {
    'mixed2' : 0.2,
    'mixed3' : 3.,
    'mixed4' : 2.,
    'mixed5' : 1.5,
} # how much layer's activation contributes to the loss you'll seek to maximize 
# check your all layer names using model.summary()

In [49]:
# 8.10 Defining the loss to be maximized

layer_dict = dict([layer.name, layer] for layer in model.layers)

loss = K.variable(0.)
for layer_name in layer_contributions:
    coeff = layer_contributions[layer_name]
    activation = layer_dict[layer_name].output
    scaling = K.prod(K.cast(K.shape(activation),'float32'))
    loss = loss + coeff * K.sum(K.square(activation[:,2:-2,2:-2,:])) # avoid border

In [54]:
# 8.11 Gradient-ascent process

dream = model.input

grads = K.gradients(loss,dream)[0]
grads /= K.maximum(K.mean(K.abs(grads)),1e-7) # Normalize the gradients
outputs = [loss,grads]
fetch_loss_and_grads = K.function([dream],outputs)

def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1] 
    return loss_value, grad_values

def gradient_ascent(x,iterations, step, max_loss = None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        #if max_loss is not None and loss_value > max_loss :
        #    break
        print('...Loss value at', i , ':', loss_value)
        x += step * grad_values
        
    return x

In [60]:
dream

<tf.Tensor 'input_1:0' shape=(?, ?, ?, 3) dtype=float32>

In [43]:
# 8.13 Auxiliary functions

import scipy
from keras.preprocessing import image

def resize_img(img, size):
    img = np.copy(img)
    factors = (1, float(size[0])/img.shape[1],float(size[1])/img.shape[2],1)
    return scipy.ndimage.zoom(img,factors,order=1) # scipy.ndimage.zoom more+

def save_img(img, fname):
    pil_img = deprocess_image(np.copy(img))
    scipy.misc.imsave(fname,pil_img)
    
def preprocess_image(image_path):
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis =0)
    img = inception_v3.preprocess_input(img)
    return img

def deprocess_image(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3,x.shape[2], x.shape[3]))
        x = x.transpose((1,2,0))
    else :
        x = x.reshape((x.shape[1],x.shape[2],3))
    x /= 2.
    x += 0.5
    x *= 255.
    x = np.clip(x,0,255).astype('uint8')
    return x

In [57]:
# 8.12 Running gradient ascent over different successive scales

import numpy as np

step = 0.01
num_octave = 3
octave_scale = 1.4
iterations = 20
max_loss = 10.
base_img_path = 'goh.jpg'

img = preprocess_image(base_img_path) # (1, 440, 720, 3)
original_shape = img.shape[1:3] # (440, 720)
successive_shapes = [original_shape]

for i in range(1, num_octave):
    shape = tuple([int(dim/(octave_scale ** i)) for dim in original_shape])
    successive_shapes.append(shape)
    
successive_shapes

[(405, 512), (289, 365), (206, 261)]

In [58]:
img.shape

(1, 405, 512, 3)

In [59]:
img[0]

array([[[-0.5921569 , -0.5686275 , -0.5137255 ],
        [-0.8666667 , -0.84313726, -0.70980394],
        [-0.7490196 , -0.7254902 , -0.5764706 ],
        ...,
        [ 0.3411765 ,  0.18431377, -0.02745098],
        [ 0.30196083,  0.1686275 , -0.06666666],
        [ 0.36470592,  0.23921573, -0.01960784]],

       [[-0.5764706 , -0.58431375, -0.34117645],
        [-0.7019608 , -0.70980394, -0.3960784 ],
        [-0.6784314 , -0.6627451 , -0.34117645],
        ...,
        [ 0.43529415,  0.27843142,  0.06666672],
        [ 0.43529415,  0.30196083,  0.082353  ],
        [ 0.33333337,  0.21568632, -0.09019607]],

       [[-0.7254902 , -0.73333335, -0.49019605],
        [-0.7882353 , -0.79607844, -0.4823529 ],
        [-0.7882353 , -0.77254903, -0.47450978],
        ...,
        [ 0.43529415,  0.27843142,  0.06666672],
        [ 0.5137255 ,  0.36470592,  0.11372554],
        [ 0.45882356,  0.32549024, -0.01176471]],

       ...,

       [[-0.29411763, -0.35686272, -0.5058824 ],
        [-0

In [52]:
successive_shapes =successive_shapes[::-1] # reorder, Reverse
successive_shapes 

[(206, 261), (289, 365), (405, 512)]

In [39]:
test_img = np.copy(img)
test_img = resize_img(test_img,(224,367))
test_img.shape

C:\Users\user\Anaconda3\lib\site-packages\scipy\ndimage\interpolation.py:616: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


(1, 224, 367, 3)

In [62]:
original_img = np.copy(img)
shrunk_original_img = resize_img(img,successive_shapes[0])

for shape in successive_shapes:
    print('Processing img shape', shape)
    img = resize_img(img,shape)
    img = gradient_ascent(img, iterations=iterations,step=step,max_loss=max_loss)
    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    same_size_original = resize_img(original_img,shape)
    lost_detail = same_size_original - upscaled_shrunk_original_img
    
    img += lost_detail
    shrunk_original_img = resize_img(original_img,shape)
    save_img(img, fname='dream_at_scale_'+str(shape)+'.png')
save_img(img, fname = 'final_dream.png')

Processing img shape (405, 512)
...Loss value at 0 : 1932302.0
...Loss value at 1 : 2743010.5
...Loss value at 2 : 3504646.8


KeyboardInterrupt: 